# Project Overview

This project focuses on building a **binary classification model** to detect **Pneumonia vs Normal** from chest X-ray images.  

We use the **Chest X-Ray and OCT Medical Image Dataset** and implement the following workflow:

1. **Data Exploration:**  
   - Inspect the dataset structure, count images per class, and visualize samples.  
   - Check class distribution and prepare for training/validation/test splits.  

2. **Data Preparation:**  
   - Apply **data augmentation** on the training set to improve model robustness.  
   - Split the original training set into **training** (80%) and **validation** (20%) sets.  
   - Normalize images based on ImageNet statistics and prepare PyTorch `DataLoaders`.  

3. **Model Setup:**  
   - Load a pre-trained **DenseNet121** model.  
   - Fine-tune the last two dense blocks and replace the classifier for binary output.  
   - Use **BCEWithLogitsLoss**, Adam optimizer, and a learning rate scheduler.  

4. **Training & Validation:**  
   - Train the model while monitoring validation loss and accuracy.  
   - Implement **early stopping** to avoid overfitting.  
   - Save the best model based on validation performance.  

5. **Testing & Evaluation:**  
   - Evaluate the best model on the test set.  
   - Compute accuracy, confusion matrix, and a full classification report. 

In [1]:
import kagglehub
import os
from collections import defaultdict
import shutil
import random
from sklearn.model_selection import train_test_split
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torchvision.models as models
from torchvision.models import DenseNet121_Weights
import copy
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Subset

In [2]:
path = kagglehub.dataset_download("gallo33henrique/chest-x-ray-and-oct-medical-image-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/chest-x-ray-and-oct-medical-image-dataset


In [3]:
base_path = path

def print_dir_tree(start_path, max_depth=3):
    for root, dirs, files in os.walk(start_path):
        depth = root.replace(start_path, "").count(os.sep)
        if depth > max_depth:
            continue
        indent = "│   " * depth + "├── "
        print(f"{indent}{os.path.basename(root)}/")
        for f in files[:5]:
            print(f"{indent}    {f}")

print("Dataset directory structure:\n")
print_dir_tree(base_path)

Dataset directory structure:

├── chest-x-ray-and-oct-medical-image-dataset/
│   ├── data/
│   │   ├── test/
│   │   │   ├── PNEUMONIA/
│   │   │   ├──     person1676_virus_2892.jpeg
│   │   │   ├──     person1650_virus_2852.jpeg
│   │   │   ├──     person22_virus_55.jpeg
│   │   │   ├──     person122_bacteria_582.jpeg
│   │   │   ├──     person85_bacteria_417.jpeg
│   │   │   ├── NORMAL/
│   │   │   ├──     NORMAL2-IM-0336-0001.jpeg
│   │   │   ├──     IM-0101-0001.jpeg
│   │   │   ├──     NORMAL2-IM-0337-0001.jpeg
│   │   │   ├──     NORMAL2-IM-0198-0001.jpeg
│   │   │   ├──     IM-0013-0001.jpeg
│   │   ├── train/
│   │   │   ├── PNEUMONIA/
│   │   │   ├──     person1180_virus_2010.jpeg
│   │   │   ├──     person1230_virus_2081.jpeg
│   │   │   ├──     person1513_virus_2632.jpeg
│   │   │   ├──     person124_virus_238.jpeg
│   │   │   ├──     person746_virus_1369.jpeg
│   │   │   ├── NORMAL/
│   │   │   ├──     NORMAL2-IM-0771-0001.jpeg
│   │   │   ├──     NORMAL2-IM-1294-0001-0002.

In [4]:
data_path = os.path.join(base_path, "data")

splits = ["train", "test"]
class_counts = defaultdict(dict)

for split in splits:
    split_path = os.path.join(data_path, split)
    for cls in os.listdir(split_path):
        cls_path = os.path.join(split_path, cls)
        if os.path.isdir(cls_path):
            num_images = len([
                f for f in os.listdir(cls_path)
                if f.lower().endswith((".jpg", ".jpeg", ".png"))
            ])
            class_counts[split][cls] = num_images

for split in class_counts:
    print(f"\n{split.upper()} SET")
    print("-" * 20)
    for cls, count in class_counts[split].items():
        print(f"{cls:<12}: {count}")
    print(f"Total images: {sum(class_counts[split].values())}")


TRAIN SET
--------------------
PNEUMONIA   : 3883
NORMAL      : 1349
Total images: 5232

TEST SET
--------------------
PNEUMONIA   : 390
NORMAL      : 234
Total images: 624


We split the original training set into **training** and **validation** subsets (80/20) because the test set should only be used for the final evaluation.  

- **Training set:** used to train the model.  
- **Validation set:** used to monitor performance and tune the model during training.  

In [5]:
original_train_path = os.path.join(data_path, "train")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

full_dataset = datasets.ImageFolder(root=original_train_path, transform=transform)

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(
    full_dataset, 
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

Training samples: 4185
Validation samples: 1047


In [6]:
dataset_splits = {
    "TRAIN": train_dataset,
    "VAL": val_dataset
}

print("Dataset verification (In-Memory Splits):\n")

for name, subset in dataset_splits.items():
    print(f"{name} SET")
    print("-" * 25)
    

    class_names = full_dataset.classes
    indices = subset.indices
    labels = [full_dataset.targets[i] for i in indices]
    
    total = 0
    for i, class_name in enumerate(class_names):
        count = labels.count(i)
        print(f"{class_name:<12}: {count}")
        total += count
    
    print(f"Total images: {total}\n")

test_path = os.path.join(data_path, "test")
print("TEST SET")
print("-" * 25)

if os.path.exists(test_path):
    test_total = 0
    for cls in os.listdir(test_path):
        cls_path = os.path.join(test_path, cls)
        if os.path.isdir(cls_path):
            num_images = len([f for f in os.listdir(cls_path) if f.lower().endswith((".jpg", ".jpeg", ".png"))])
            print(f"{cls:<12}: {num_images}")
            test_total += num_images
    print(f"Total images: {test_total}\n")
else:
    print("❌ Test folder not found!\n")

print("Verification complete ✅")

Dataset verification (In-Memory Splits):

TRAIN SET
-------------------------
NORMAL      : 1079
PNEUMONIA   : 3106
Total images: 4185

VAL SET
-------------------------
NORMAL      : 270
PNEUMONIA   : 777
Total images: 1047

TEST SET
-------------------------
PNEUMONIA   : 390
NORMAL      : 234
Total images: 624

Verification complete ✅


In [7]:
from torchvision.models import densenet121
model = densenet121()
#print(model)


**Prepares the dataset and dataloaders for training, validation, and testing:**

1. **Settings:** defines image size, batch size, number of workers, and random seed for reproducibility.  
2. **Transforms:**  
   - `train_transforms` applies data augmentation (random crop, flip, rotation, color jitter) to make the model more robust.  
   - `val_test_transforms` resizes and normalizes images without augmentation for validation and testing.  
3. **Dataset split:** the original training data is split into **train** (80%) and **validation** (20%) subsets using a fixed random seed.  
4. **DataLoaders:** wrap the datasets for efficient batch loading during training and evaluation.  

In [8]:
IMAGE_SIZE = 224
BATCH_SIZE = 32
NUM_WORKERS = 2  
SEED = 42
torch.manual_seed(SEED)

imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std  = [0.229, 0.224, 0.225]


train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
])

val_test_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
])


full_train_data = datasets.ImageFolder(os.path.join(data_path, "train"), transform=train_transforms)
full_val_data   = datasets.ImageFolder(os.path.join(data_path, "train"), transform=val_test_transforms)


num_train = len(full_train_data)
indices = list(range(num_train))
split = int(np.floor(0.2 * num_train)) 

np.random.seed(SEED)
np.random.shuffle(indices)

train_idx, val_idx = indices[split:], indices[:split]


train_dataset = torch.utils.data.Subset(full_train_data, train_idx)
val_dataset   = torch.utils.data.Subset(full_val_data, val_idx)


test_dataset  = datasets.ImageFolder(os.path.join(data_path, "test"), transform=val_test_transforms)


train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

class_names = full_train_data.classes
print("Classes:", class_names)
print(f"Train samples: {len(train_dataset)}")
print(f"Val samples:   {len(val_dataset)}")
print(f"Test samples:  {len(test_dataset)}")
print("DataLoaders ready ✅")

Classes: ['NORMAL', 'PNEUMONIA']
Train samples: 4186
Val samples:   1046
Test samples:  624
DataLoaders ready ✅


**Set up the model, loss, and optimizer for training:**

1. **Device:** uses GPU if available for faster training.  
2. **Model:** loads a pre-trained **DenseNet121**.  
   - Only the last two dense blocks (`denseblock3` and `denseblock4`) are fine-tuned; earlier layers are frozen.  
   - The classifier is replaced with a **dropout + linear layer** for binary classification.  
3. **Loss & Optimizer:** uses `BCEWithLogitsLoss` and Adam optimizer (only trainable parameters).  
4. **Scheduler:** reduces learning rate if validation loss plateaus.  

The model is then moved to the selected device and ready for training.


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = models.densenet121(weights=DenseNet121_Weights.IMAGENET1K_V1)


for name, param in model.named_parameters():
    if "denseblock3" in name or "denseblock4" in name:
        param.requires_grad = True


num_features = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(num_features, 1)  
)

model = model.to(device)

criterion = nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.3, patience=3, min_lr=1e-7
)

print("DenseNet121 model ready ✅")


Using device: cuda
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


100%|██████████| 30.8M/30.8M [00:00<00:00, 165MB/s]


DenseNet121 model ready ✅


1. **Training & Validation functions:**  
   - `train_one_epoch` runs a full pass over the training data, computing loss and accuracy, and updating model weights.  
   - `validate` evaluates the model on the validation set without updating weights.  


2. **Training loop:**  
   - Runs for a maximum of `MAX_EPOCHS` epochs.  
   - After each epoch, the validation loss is monitored.  
   - Learning rate is reduced if validation loss plateaus.  
   - The model is saved whenever validation loss improves.  


3. **Early stopping:** stops training if validation loss does not improve for `PATIENCE` consecutive epochs.  

Finally, the best model (with lowest validation loss) is loaded for further evaluation.


In [10]:
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(loader, leave=False):
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)  

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = (torch.sigmoid(outputs) > 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total


def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(loader, leave=False):
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            preds = (torch.sigmoid(outputs) > 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total


MAX_EPOCHS = 25
PATIENCE = 5

best_val_loss = float("inf")
epochs_no_improve = 0
best_model_wts = copy.deepcopy(model.state_dict())

for epoch in range(MAX_EPOCHS):
    print(f"\nEpoch {epoch+1}/{MAX_EPOCHS}")
    print("-" * 30)

    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.4f}")

    scheduler.step(val_loss)
    old_lr = optimizer.param_groups[0]['lr']
    scheduler.step(val_loss)
    new_lr = optimizer.param_groups[0]['lr']
    if new_lr != old_lr:
        print(f"🔻 LR reduced: {old_lr:.2e} -> {new_lr:.2e}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save(best_model_wts, "/kaggle/working/best_model.pth")
        epochs_no_improve = 0
        print("✅ Validation loss improved — saving model")
    else:
        epochs_no_improve += 1
        print(f"⚠️ No improvement for {epochs_no_improve} epoch(s)")

    if epochs_no_improve >= PATIENCE:
        print("\n⏹️ Early stopping triggered")
        break

model.load_state_dict(best_model_wts)
torch.save(model.state_dict(), "/kaggle/working/best_model_final.pth")
print("\nBest model loaded ✅")



Epoch 1/25
------------------------------


Train Loss: 0.1908 | Train Acc: 0.9271
Val   Loss: 0.0714 | Val   Acc: 0.9771
✅ Validation loss improved — saving model

Epoch 2/25
------------------------------


Train Loss: 0.0945 | Train Acc: 0.9694
Val   Loss: 0.0475 | Val   Acc: 0.9837
✅ Validation loss improved — saving model

Epoch 3/25
------------------------------


Train Loss: 0.0827 | Train Acc: 0.9687
Val   Loss: 0.0530 | Val   Acc: 0.9790
⚠️ No improvement for 1 epoch(s)

Epoch 4/25
------------------------------


Train Loss: 0.0636 | Train Acc: 0.9775
Val   Loss: 0.0427 | Val   Acc: 0.9876
✅ Validation loss improved — saving model

Epoch 5/25
------------------------------


...

Train Loss: 0.0304 | Train Acc: 0.9907
Val   Loss: 0.0452 | Val   Acc: 0.9857
⚠️ No improvement for 1 epoch(s)

Epoch 9/25
------------------------------


Train Loss: 0.0249 | Train Acc: 0.9912
Val   Loss: 0.0463 | Val   Acc: 0.9866
⚠️ No improvement for 2 epoch(s)

Epoch 10/25
------------------------------


Train Loss: 0.0190 | Train Acc: 0.9945
Val   Loss: 0.0431 | Val   Acc: 0.9885
⚠️ No improvement for 3 epoch(s)

Epoch 11/25
------------------------------


Train Loss: 0.0173 | Train Acc: 0.9955
Val   Loss: 0.0401 | Val   Acc: 0.9895
⚠️ No improvement for 4 epoch(s)

Epoch 12/25
------------------------------


Train Loss: 0.0166 | Train Acc: 0.9945
Val   Loss: 0.0403 | Val   Acc: 0.9895
⚠️ No improvement for 5 epoch(s)

⏹️ Early stopping triggered

Best model loaded ✅


Evaluate the **trained model** on the test set

In [11]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)

        outputs = model(images)
        probs = torch.sigmoid(outputs)
        preds = (probs > 0.5).float()

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

all_preds = np.array(all_preds).flatten()
all_labels = np.array(all_labels).flatten()

test_acc = (all_preds == all_labels).mean()
print(f"Test Accuracy: {test_acc:.4f}")

cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:\n", cm)

print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=class_names))


Test Accuracy: 0.9263
Confusion Matrix:
 [[189  45]
 [  1 389]]
Classification Report:
              precision    recall  f1-score   support

      NORMAL       0.99      0.81      0.89       234
   PNEUMONIA       0.90      1.00      0.94       390

    accuracy                           0.93       624
   macro avg       0.95      0.90      0.92       624
weighted avg       0.93      0.93      0.92       624

